In [39]:
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
import requests
import QuantLib as ql
import numpy as np
import matplotlib.pyplot as plt
import json
from sklearn.metrics import mean_squared_error
from scipy.optimize import differential_evolution
import concurrent.futures
from tradingview_screener import Query, col
import rookiepy
from sklearn.preprocessing import MinMaxScaler
import multiprocessing as mp


with open("config.json", "r") as config_file:
    config = json.load(config_file)

api_key = config.get("api_key")
secret_key = config.get("secret_key")

pd.set_option('display.max_rows', None)


In [40]:
NASDAQ = pd.read_csv('Indexes/NASDAQ.csv')
DOWJ = pd.read_csv('Indexes/DOWJ.csv')
SP = pd.read_csv('Indexes/S&P500.csv')

def clean_data(df):
    df = df[['Company', 'Symbol']]
    df = pd.DataFrame(df).dropna()
    return df

NASDAQ = clean_data(NASDAQ)
DOWJ = clean_data(DOWJ)
SP = clean_data(SP)


In [43]:
    
#TODO FIX SCREENER 
def screen_stocks():
    # Get cookies for TradingView session
    cookies = rookiepy.to_cookiejar(rookiepy.chrome(['.tradingview.com']))
    
    # Fetch stock data
    _, df = Query().select('close','change', 'Perf.3M').where(
        col('close').between(20, 55),
        col('change').between(-4,-2),
        col('Perf.3M') > 0,
        col('exchange').isin(['AMEX', 'CBOE', 'NASDAQ', 'NYSE']),

        ).limit(1000).get_scanner_data(cookies=cookies)
    
    df[['exchange', 'ticker']] = df['ticker'].str.split(':', expand=True)
    
    return df


def get_rolling_price_change_avg(ticker: str, days: int):
    try:
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days+10)
        
        data = yf.download(ticker, start=start_date, end=end_date, progress=False)
        
        if data.empty:
            return None, None
        
        data = data.sort_index()
        current_price = get_current_stock_price(ticker)

        data['Price_Change'] = ((current_price - data['Close'].shift(1)) / data['Close'].shift(1)) * 100

        rolling_avg = data['Price_Change'].rolling(window=min(days, len(data))).mean().iloc[-1]

        return rolling_avg
    
    except Exception as e:
        print(f"Error occurred for ticker {ticker}: {e}")
        return None, None

def get_current_stock_price(symbol: str):

    url = "https://data.alpaca.markets/v2/stocks/trades/latest"

    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": api_key,
        "APCA-API-SECRET-KEY": secret_key,
    }

    params = {
        "symbols": symbol,  
        "feed": "iex" 
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  

        data = response.json()
        return data.get("trades", {}).get(symbol, {}).get("p") 

    except requests.exceptions.RequestException as e:
        print(f"Error fetching stock price: {e}")


def get_option_chain(api_key: str, secret_key: str, ticker: str, expiration_date: datetime):
    expiration_str = expiration_date.strftime("%Y-%m-%d")  # Convert datetime to string
    
    url = f"https://data.alpaca.markets/v1beta1/options/snapshots/{ticker}?feed=indicative&limit=100&expiration_date={expiration_str}"
    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": api_key,
        "APCA-API-SECRET-KEY": secret_key,
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
        option_chain = data.get("snapshots", {})

        if not option_chain:
            return None

        parsed_data = []
        for symbol, details in option_chain.items():
            expiration_start = len(symbol) - 15
            option_type = "Call" if symbol[expiration_start+6] == "C" else "Put"
            strike_price = int(symbol[expiration_start+7:]) / 1000  

            greeks = details.get("greeks", {}) or {}
            latest_quote = details.get("latestQuote", {})

            parsed_data.append({
                "symbol": ticker,
                "expiration_date": expiration_str,  # Use the formatted string
                "option_type": option_type,
                "strike_price": strike_price,
                "delta": greeks.get("delta"),
                "gamma": greeks.get("gamma"),
                "rho": greeks.get("rho"),
                "theta": greeks.get("theta"),
                "vega": greeks.get("vega"),
                "implied_volatility": details.get("impliedVolatility"),
                "ask_price": latest_quote.get("ap"),
                "ask_size": latest_quote.get("as"),
                "bid_price": latest_quote.get("bp"),
                "bid_size": latest_quote.get("bs"),
            })

        return pd.DataFrame(parsed_data)

    except requests.exceptions.RequestException as e:
        print(f"Error fetching option chain: {e}")
        return None
    
    

def gbm(s0, mu, sigma, deltaT, dt):
    """
    Models a stock price S(t) using the Wiener process W(t) as
    `S(t) = S(0).exp{(mu-(sigma^2/2).t)+sigma.W(t)}`
    
    Arguments:
        s0: Initial stock price, default 100
        mu: 'Drift' of the stock (upwards or downwards), default 0.2
        sigma: 'Volatility' of the stock, default 0.68
        deltaT: The time period for which the future prices are computed, default 52 (as in 52 weeks)
        dt: The granularity of the time-period, default 0.1
    
    Returns:
        time_vector: array of time steps
        s: array with the simulated stock prices over the time-period deltaT
    """
    n_step = int(deltaT / dt)  # Number of time steps
    time_vector = np.linspace(0, deltaT, num=n_step)  # Time vector
    
    # Wiener process: cumulative sum of random normal increments
    random_increments = np.random.normal(0, np.sqrt(dt), size=n_step)
    weiner_process = np.cumsum(random_increments)
    
    # Stock price simulation
    stock_var = (mu - (sigma**2 / 2)) * time_vector
    s = s0 * np.exp(stock_var + sigma * weiner_process)
    
    return s


def objective(params, real_prices, s0):
    """Objective function for optimization."""
    mu, sigma = params  # Unpack parameters
    gbm_prices = gbm(s0, mu, sigma, deltaT=len(real_prices), dt=1)
    return mean_squared_error(real_prices, gbm_prices)

def optimize_gbm(symbol: str, training_period: str, bin_length: int):
    """
    Optimize μ and σ over multiple time bins, weighting recent periods more.
    """
    # Fetch real stock data (past 5 years)
    stock_data = yf.download(symbol, period=training_period, interval="1d")
    real_prices = stock_data["Close"].dropna().values

    num_bins = len(real_prices) // bin_length
    weights = np.linspace(1, 2, num_bins)  # Increasing weights for recent bins

    mu_values, sigma_values, mses = [], [], []

    for i in range(num_bins):
        bin_prices = real_prices[i * bin_length : (i + 1) * bin_length]
        s0 = bin_prices[0]

        bounds = [(-0.3, 0.3), (0.001, 0.35)]

        result = differential_evolution(objective, bounds, args=(bin_prices, s0))
        best_mu, best_sigma = result.x
        best_mse = result.fun

        mu_values.append(best_mu)
        sigma_values.append(best_sigma)
        mses.append(best_mse)

    weight_sum = np.sum(weights)
    avg_mu = np.sum(np.array(mu_values) * weights) / weight_sum
    avg_sigma = np.sum(np.array(sigma_values) * weights) / weight_sum

    print(f"\nFinal Weighted Averages: μ = {avg_mu:.4f}, σ = {avg_sigma:.4f}")

    return avg_mu, avg_sigma


def compute_score(contracts):
    """Compute the weighted score for contracts using normalized values."""
    # Create a copy to avoid modifying the original data
    temp_contracts = contracts.copy()
    
    # Normalize only for scoring
    scaler = MinMaxScaler()
    temp_contracts[['profitability_percent', 'percent_return', 'sharpe_ratio']] = scaler.fit_transform(
        temp_contracts[['profitability_percent', 'percent_return', 'sharpe_ratio']]
    )
    
    # Compute score using normalized values
    temp_contracts['score'] = (
        0.50 * temp_contracts['profitability_percent'] +
        0.25 * temp_contracts['percent_return'] +
        0.25 * temp_contracts['sharpe_ratio']
    )
    
    # Merge only the score back to the original DataFrame
    contracts['score'] = temp_contracts['score']
    
    return contracts

def select_optimal_contract(contracts):
    """Select the contract with the highest weighted score while keeping original values."""
    contracts = compute_score(contracts)
    contracts = contracts.sort_values(by='score', ascending=False)
    return contracts

def audit_contracts(expiration_date: datetime):
    # t-bill 3-month rate: 4.19%, inflation rate: 2.9% -> scaled to weekly
    risk_free_rate = (((1 + 0.0419) / (1 + 0.029)) ** (1/52) - 1) * 100
    simulation_attempts = 250
    optimizer_training_period = "2y"
    bin_length = 20

    all_options = pd.DataFrame(columns=[
        'symbol', 'expiration_date', 'option_type', 'strike_price', 'delta', 
        'gamma', 'rho', 'theta', 'vega', 'implied_volatility', 'ask_price', 
        'ask_size', 'bid_price', 'bid_size'
    ])
    
    candidates = screen_stocks()['ticker'].to_list()[0:2]
    
    print(candidates)

    for symbol in candidates:
        print(f"Evaluating {symbol}")
        option_chain = get_option_chain(api_key=api_key, secret_key=secret_key, ticker=symbol, expiration_date=expiration_date)

        if option_chain is None or option_chain.empty:
            continue 
        
        put_chain = option_chain[
            (option_chain['option_type'] == 'Put') & (option_chain['rho'].notna())
        ].sort_values(by='strike_price', ascending=True)

        price = get_current_stock_price(symbol)
        optimized_mu, optimized_sigma = optimize_gbm(symbol=symbol, training_period=optimizer_training_period, bin_length=bin_length)

        profitability_chances = []
        percent_returns = []

        for index, contract in put_chain.iterrows():
            count = 0
            strike_price = contract['strike_price']
            bid_price = contract['bid_price']
            ask_price = contract['ask_price']
            mid_price = (bid_price + ask_price) / 2

            # Skew the assumed fill price towards the bid
            fill_price = 0.75 * bid_price + 0.25 * mid_price
            premium_collected = fill_price * 100  # Since options are in 100-share contracts

            for _ in range(simulation_attempts):
                prices = gbm(
                    s0=price, mu=optimized_mu, sigma=optimized_sigma, 
                    deltaT=np.busday_count(datetime.today().date(), datetime.strptime(contract['expiration_date'], "%Y-%m-%d").date()), 
                    dt=1
                )
            
                if prices[-1] > strike_price:
                    count += 1

            profitability_chance = (count / simulation_attempts) * 100
            percent_return = (premium_collected / (strike_price * 100)) * 100  # Return as a percentage of collateral

            profitability_chances.append(profitability_chance)
            percent_returns.append(percent_return)

        put_chain['profitability_percent'] = profitability_chances
        put_chain['percent_return'] = percent_returns
        put_chain['expected_value'] = put_chain['profitability_percent'] * put_chain['percent_return']
        put_chain['current_price'] = price

        if put_chain['percent_return'].std() != 0:
            put_chain['sharpe_ratio'] = (put_chain['percent_return'] - risk_free_rate) / put_chain['percent_return'].std()
        else:
            put_chain['sharpe_ratio'] = 0  # Avoid division by zero

        all_options = pd.concat([all_options, put_chain], ignore_index=True, copy=False)

    return all_options


In [44]:

priced_contracts = audit_contracts(expiration_date = datetime(year=2025, month=2, day=14))

['ARCC', 'BITX']
Evaluating ARCC
Evaluating BITX


[*********************100%***********************]  1 of 1 completed



Final Weighted Averages: μ = 0.0081, σ = 0.0345


/var/folders/90/x6spqd2j4cbb38yrckqjm8840000gn/T/ipykernel_1671/1095512022.py:294: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_options = pd.concat([all_options, put_chain], ignore_index=True, copy=False)


In [45]:
contracts = select_optimal_contract(priced_contracts)
print('length',len(contracts))
display(contracts)

length 16


,symbol,expiration_date,option_type,strike_price,delta,gamma,rho,theta,vega,implied_volatility,ask_price,ask_size,bid_price,bid_size,profitability_percent,percent_return,expected_value,current_price,sharpe_ratio,score
13,BITX,2025-02-14,Put,51.5,-0.3348,0.0430,-0.0050,-0.1689,0.0311,0.9926,2.20,319,1.89,1,92.0,3.745146,344.553398,54.3,3.550474,0.711024
10,BITX,2025-02-14,Put,50.0,-0.2666,0.0394,-0.0039,-0.1505,0.0280,0.9787,1.51,8,1.43,102,95.2,2.880000,274.176000,54.3,2.725017,0.664148
11,BITX,2025-02-14,Put,50.5,-0.2888,0.0408,-0.0043,-0.1569,0.0291,0.9820,1.71,1,1.58,102,92.0,3.160891,290.801980,54.3,2.993023,0.624239
12,BITX,2025-02-14,Put,51.0,-0.3084,0.0429,-0.0046,-0.1581,0.0300,0.9612,1.83,23,1.69,1,90.8,3.348039,304.001961,54.3,3.171585,0.621426
8,BITX,2025-02-14,Put,49.0,-0.2279,0.0357,-0.0034,-0.1405,0.0258,0.9928,1.26,11,1.16,1,96.0,2.392857,229.714286,54.3,2.260222,0.612196
4,BITX,2025-02-14,Put,47.0,-0.1642,0.0280,-0.0024,-0.1204,0.0211,1.0364,0.89,20,0.77,17,98.8,1.670213,165.017021,54.3,1.570729,0.576282
3,BITX,2025-02-14,Put,46.0,-0.1408,0.0244,-0.0021,-0.1127,0.0191,1.0735,0.76,11,0.66,1,100.0,1.461957,146.195652,54.3,1.372027,0.575960
7,BITX,2025-02-14,Put,48.5,-0.2106,0.0337,-0.0031,-0.1359,0.0246,1.0042,1.19,11,1.02,1,96.0,2.146907,206.103093,54.3,2.025555,0.575662
5,BITX,2025-02-14,Put,47.5,-0.1779,0.0299,-0.0026,-0.1248,0.0222,1.0215,0.97,7,0.84,50,98.0,1.802632,176.657895,54.3,1.697073,0.575544
6,BITX,2025-02-14,Put,48.0,-0.1937,0.0318,-0.0028,-0.1304,0.0234,1.0124,1.03,5,0.97,108,96.4,2.036458,196.314583,54.3,1.920173,0.569460


In [ ]:
canidates = screen_stocks()['ticker'].to_list()
canidates

['SOXS',
 'BITX',
 'NXT',
 'BITU',
 'CHWY',
 'UCO',
 'PGNY',
 'SQM',
 'DXYZ',
 'QFIN',
 'UTI',
 'MATW',
 'TBT',
 'MAGX',
 'WEBL',
 'EDN',
 'SPH',
 'ARQQ',
 'AAPB',
 'QQQU',
 'ODD',
 'DRV',
 'ALGS',
 'DXF',
 'EXOD',
 'IPI',
 'AGFY',
 'RCKY',
 'TESL',
 'UCC',
 'DRAG',
 'NYAX',
 'GBLI',
 'PFX',
 'UNB',
 'TFSA',
 'IROQ',
 'EVAV',
 'CNFRZ',
 'TSLW',
 'BANFP',
 'FGF',
 'UX',
 'BTRN']

In [ ]:
# TODO
"""
- Figure out way to normalize stock price, whether that is min max of the range of the price(shoudl help optimizer
- Find better metric for optimizer
- Try binning, so like get past 10 years of AAPL, seperate into bins of 20 or n trading days, train optimzer on each one. Then the hyperparameters can be weighted to have more bias towards more recent bins
"""




'\n- Figure out way to normalize stock price, whether that is min max of the range of the price(shoudl help optimizer\n- Find better metric for optimizer\n- Try binning, so like get past 10 years of AAPL, seperate into bins of 20 or n trading days, train optimzer on each one. Then the hyperparameters can be weighted to have more bias towards more recent bins\n'

In [ ]:
# Archive

    # def filter_stocks(rolling_change_period): 
    # filtered_stocks = set()
    # stocks = screen_stocks()

    # for index, stock in stocks.iterrows():
    #     try:
    #         today_change, rolling_avg = get_rolling_price_change_avg(stock['ticker'], days=rolling_change_period)
    #         current_price = get_current_stock_price(stock['ticker'])

    #         # Skip if any value is None
    #         if None in (today_change, rolling_avg, current_price):
    #             print(f"Skipping {stock['ticker']} due to missing data.")
    #             continue

    #         # Apply filtering conditions
    #         if (rolling_avg > 0.00): 
    #             filtered_stocks.add(stock['ticker'])

    #     except Exception as e:
    #         print(f"Skipping {stock['ticker']} due to error: {e}")
    #         continue
    
    # return filtered_stocks

    #def ORIGINAL_LOGIC_FOR _AUDITING_OPTIONS()
    # simulation_attempts = 200
    # optimizer_training_period = "2y"
    # bin_length = 20
    # rolling_change_period = 15
    # expiration_date = datetime(year=2025, month=2, day=14) 
    # all_options = pd.DataFrame(columns=['symbol', 'expiration_date', 'option_type', 'strike_price', 'delta', 'gamma', 'rho', 'theta', 'vega', 'implied_volatility', 'ask_price', 'ask_size', 'bid_price', 'bid_size'])
    # candidates = ["AAPL", "AMD"]
    # # filter_stocks(rolling_change_period=rolling_change_period)

    # # t-bill 3-month rate: 4.19%, inflation rate: 2.9% -> scaled to weekly
    # risk_free_rate = (((1 + 0.0419) / (1 + 0.029)) ** (1/52) - 1) * 100

    # print(candidates)

    # for symbol in candidates:
    #     option_chain = get_option_chain(api_key=api_key, secret_key=secret_key, ticker=symbol, expiration_date=expiration_date)
    #     put_chain = option_chain[(option_chain['option_type'] == 'Put') & (option_chain['rho'].notna())].sort_values(by='strike_price', ascending=True)

    #     if option_chain is None or option_chain.empty:
    #         continue 

    #     price = get_current_stock_price(symbol)
    #     optimized_mu, optimized_sigma = optimize_gbm(symbol=symbol, training_period=optimizer_training_period, bin_length=bin_length)

    #     profitability_chances = []
    #     percent_returns = []

    #     for index, contract in put_chain.iterrows():
    #         count = 0
    #         strike_price = contract['strike_price']

    #         for i in range(simulation_attempts):
    #             prices = gbm(s0=price, mu=optimized_mu, sigma=optimized_sigma, 
    #                 deltaT=np.busday_count(datetime.today().date(), datetime.strptime(contract['expiration_date'], "%Y-%m-%d").date()), dt=1)  
    #             if prices[-1] > strike_price:
    #                 count += 1
    #         profitability_chance = (count / simulation_attempts) * 100
    #         profit = (contract['bid_price']*contract['bid_size'] + contract['ask_price']*contract['ask_size']) / (contract['ask_size'] + contract['bid_size'])
    #         percent_return = (profit / (strike_price)) * 100

    #         profitability_chances.append(profitability_chance)
    #         percent_returns.append(percent_return)
    #     put_chain['profitability_percent'] = profitability_chances
    #     put_chain['percent_return'] = percent_returns
    #     put_chain['expected_value'] = put_chain['profitability_percent'] * put_chain['percent_return']
    #     put_chain['current_price'] = price
    #     if put_chain['percent_return'].std() != 0:
    #         put_chain['sharpe_ratio'] = (put_chain['percent_return'] - risk_free_rate) / put_chain['percent_return'].std()
    #     else:
    #         put_chain['sharpe_ratio'] = 0  # Avoid division by zero
    #     all_options = pd.concat([all_options, put_chain], ignore_index=True, copy=False)


# def gbm_vs_real_graph(symbol, mu, sigma, period):
#     stock_data = yf.download(symbol, period=period, interval="1d")
#     real_prices = stock_data["Close"].dropna().values
#     time_steps = np.arange(len(real_prices))


#     gbm_path = gbm(s0 = real_prices[0], mu=mu, sigma=sigma, deltaT=len(real_prices), dt=1)
#     plt.figure(figsize=(10, 5))
#     plt.plot(time_steps, real_prices, label="Real Prices", color="blue")
#     plt.plot(time_steps, gbm_path, label="GBM Simulated", linestyle="dashed", color="red")
    
#     plt.xlabel("Time (Days)")
#     plt.ylabel("Price")
#     plt.title(f"GBM vs Real Prices for {symbol}")
#     plt.legend()
#     plt.grid()
#     plt.show()

# def multithread_optimize_bin(bin_prices, bin_size, weights, i):
#     s0 = bin_prices[0]

#     # Define the bounds for optimization
#     bounds = [(-0.3, 0.3), (0.001, 0.30)]

#     # Run the optimizer for the bin
#     result = differential_evolution(objective, bounds, args=(bin_prices, s0))
#     best_mu, best_sigma = result.x
#     best_mse = result.fun

#     print(f"Bin {i+1}: μ = {best_mu:.4f}, σ = {best_sigma:.4f}, MSE = {best_mse:.4f}")
#     return best_mu, best_sigma, best_mse

# def multithread_optimize_gbm(symbol): 
    # """
    # Optimize μ and σ over multiple time bins, weighting recent periods more.
    # """
    # # Fetch real stock data (past 2 years)
    # stock_data = yf.download(symbol, period="2y", interval="1d")
    # real_prices = stock_data["Close"].dropna().values

    # # Split into bins of 20 trading days
    # bin_size = 20
    # num_bins = len(real_prices) // bin_size
    # weights = np.linspace(1, 2, num_bins)  # Increasing weights for recent bins

    # # Initialize containers for results
    # mu_values, sigma_values, mses = [], [], []

    # # Use concurrent.futures for parallel processing of bins
    # with concurrent.futures.ThreadPoolExecutor() as executor:
    #     futures = []
    #     for i in range(num_bins):
    #         bin_prices = real_prices[i * bin_size : (i + 1) * bin_size]
    #         futures.append(executor.submit(optimize_bin, bin_prices, bin_size, weights, i))
        
    #     for future in concurrent.futures.as_completed(futures):
    #         best_mu, best_sigma, best_mse = future.result()
    #         mu_values.append(best_mu)
    #         sigma_values.append(best_sigma)
    #         mses.append(best_mse)

    # # Compute weighted averages
    # weight_sum = np.sum(weights)
    # avg_mu = np.sum(np.array(mu_values) * weights) / weight_sum
    # avg_sigma = np.sum(np.array(sigma_values) * weights) / weight_sum

    # print(f"\nFinal Weighted Averages: μ = {avg_mu:.4f}, σ = {avg_sigma:.4f}")

    # return avg_mu, avg_sigma